# **The Turing Apex Challenge**

Runtime ---> Change runtime type ---> T4 GPU

In [1]:
# ## 1. installing necessary packages
!pip install -q transformers torch accelerate bitsandbytes pandas wikipedia arxiv sentence-transformers faiss-cpu
# Imports
import os
import time
import re
import random
import wikipedia
import arxiv
import warnings
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
from tqdm.auto import tqdm

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ## 2. fetching from Wiki and arXiv

wikipedia.set_lang("en")
warnings.filterwarnings("ignore", category=UserWarning, module="wikipedia")

def fetch_wiki_context(query: str, sentences: int = 5) -> str:
    time.sleep(0.5)
    try:
        results = wikipedia.search(query)
    except Exception:
        return ""
    if not results:
        return ""

    for title in results:
        try:
            page = wikipedia.page(title, auto_suggest=False, redirect=True)
            lines = page.summary.split("\n")
            if lines:
                return " ".join(lines[:sentences])
        except wikipedia.DisambiguationError as e:
            for opt in e.options:
                try:
                    page = wikipedia.page(opt, auto_suggest=False, redirect=True)
                    lines = page.summary.split("\n")
                    if lines:
                        return " ".join(lines[:sentences])
                except Exception:
                    continue
        except Exception:
            continue

    return ""

# I preferred just using Wikipedia context but
# I kept implementation of fetching arXiv and combined context
def fetch_arxiv_context(query: str, max_results: int = 1) -> str:
    try:
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.Relevance
        )
        # Generator yielding Result objects
        abstracts = [
            result.summary.replace('\n', ' ')
            for result in search.results()
        ]
        return " ".join(abstracts)
    except Exception as e:
        print(f"arXiv fetch error: {e}")
        return ""

def fetch_combined_context(query: str, wiki_sentences: int = 5, arxiv_results: int = 1) -> str:
    wiki_context = fetch_wiki_context(query, sentences=wiki_sentences)
    arxiv_context = fetch_arxiv_context(query, max_results=arxiv_results)
    return f"{wiki_context} {arxiv_context}"

# 2.2 clean answer output
def extract_choice(output: str) -> str:
    match = re.search(r"([A-E])", output.upper())
    return match.group(1) if match else "C"

In [3]:
# ## 3. Load Data

TRAIN_PATH = 'train_data.csv'
TEST_PATH  = 'test_data.csv'

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

In [4]:
# ## 4. Build Embedding Index for Retrieval

# 4.1 Initialize encoder and compute embeddings
#encoder = SentenceTransformer('all-mpnet-base-v2')
encoder = SentenceTransformer('all-MiniLM-L6-v2')
train_embeddings = encoder.encode(train_df['prompt'].tolist(), normalize_embeddings=True)

# 4.2 Build FAISS index
dim = train_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(dim)
faiss_index.add(train_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ## 5. Load LLM for Generation

MODEL_NAME = 'microsoft/Phi-4-reasoning-plus'

# Quantization in 4-bit to fit in Colab
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
    llm_int8_enable_fp32_cpu_offload=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config,
    device_map='auto'
)

In [ ]:
# ## 6. Retrieval + Prompt Construction

def retrieve_similar(prompt: str, k: int = 3) -> pd.DataFrame:
    emb = encoder.encode(prompt, normalize_embeddings=True)
    scores, indices = faiss_index.search(np.array([emb]), k)
    return train_df.iloc[indices[0]].reset_index(drop=True)


def build_prompt(test_row, retrieved, test_context):
    # 1. Role & task
    base = (
        "You are an expert scientific assistant. Answer multiple-choice questions concisely. "
        "Return ONLY the single letter A-E."
    )
    prompt = base + "\n\n"
    # 2. Few-shot examples
    for _, row in retrieved.iterrows():
        prompt += f"Context: {row['context']}\n"
        prompt += f"Question: {row['prompt']}\n"
        for opt in ['A','B','C','D','E']:
            prompt += f"{opt}) {row[opt]}\n"
        prompt += f"Answer: {row['answer']}\n\n"
    # 3. Test input
    prompt += f"Use Your knowledge and also check this context."
    prompt += f"Choose the best answer. Context: {test_context}\nQuestion: {test_row['prompt']}\n"
    for opt in ['A','B','C','D','E']:
        prompt += f"{opt}) {test_row[opt]}\n"
    prompt += "Answer: "
    return prompt

In [ ]:
# ## 7. Precompute contexts for training
contexts = []
for prompt in tqdm(train_df['prompt'], desc="Fetching wiki contexts"):
    contexts.append(fetch_wiki_context(prompt))

train_df['context'] = contexts

In [8]:
# ## 8. Inference on Test Set
answers = []
for _,row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
    ctx    = fetch_wiki_context(row['prompt'])
    sim    = retrieve_similar(row['prompt'], k=3)
    prompt = build_prompt(row, sim, ctx)

    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    out    = model.generate(
        inputs.input_ids,
        max_new_tokens=1,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    text   = tokenizer.decode(
        out[0][inputs.input_ids.shape[-1]:],
        skip_special_tokens=True
    )
    answers.append(extract_choice(text))

Inference:   0%|          | 0/150 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
# ## 9. Save Submission
submission = pd.DataFrame({'answer': answers})
submission.to_csv('submission.csv', index=False)
print("Submission saved.")